In [8]:
import os
from elasticsearch import Elasticsearch

ELASTIC_HOST = os.getenv('ELASTIC_HOST', 'http://localhost:9200')
ELASTIC_API_KEY_ID = os.getenv('ELASTIC_API_KEY_ID', '')
ELASTIC_API_KEY_VALUE = os.getenv('ELASTIC_API_KEY_VALUE', '')
ELASTIC_SEARCH_INDEX = os.getenv('ELASTIC_SEARCH_INDEX', 'farm-products')


In [22]:
client = Elasticsearch(
            [ELASTIC_HOST], 
            api_key=(ELASTIC_API_KEY_ID, ELASTIC_API_KEY_VALUE)
        )

In [35]:
# Delete Index
client.indices.delete(index=ELASTIC_SEARCH_INDEX, ignore=[400, 404])

/var/folders/8h/pn2qk1n5571cghfklz699nh80000gn/T/ipykernel_11144/1405706728.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  client.indices.delete(index=ELASTIC_SEARCH_INDEX, ignore=[400, 404])
/var/folders/8h/pn2qk1n5571cghfklz699nh80000gn/T/ipykernel_11144/1405706728.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.indices.delete(index=ELASTIC_SEARCH_INDEX, ignore=[400, 404])


ObjectApiResponse({'acknowledged': True})

In [36]:
# Index the document
from datetime import datetime
from elasticsearch import helpers
import uuid

docs = [
    {
        '_index': ELASTIC_SEARCH_INDEX,
        '_type': 'FARM_PRODUCT',
        '_id': uuid.uuid4(),
        'type': 'nutrition',
        'title': '24 CAROT SONA',
        'price': 100,
        'timestamp': datetime.now(),
    },
    {   
        '_index': ELASTIC_SEARCH_INDEX,
        '_type': 'FARM_PRODUCT',
        '_id': uuid.uuid4(),
        'type': 'seed',
        'title': 'CABBAGE',
        'price': 80,
        'timestamp': datetime.now(),
    },
    {
        '_index': ELASTIC_SEARCH_INDEX,
        '_type': 'FARM_PRODUCT',
        '_id': uuid.uuid4(),
        'type': 'seed',
        'title': 'CABBAGE - BETTER',
        'price': 100,
        'timestamp': datetime.now(),
    },
]
#client.index(index=ELASTIC_SEARCH_INDEX, id=1, document=doc)

helpers.bulk(client, docs, chunk_size=1000, request_timeout=200)

/var/folders/8h/pn2qk1n5571cghfklz699nh80000gn/T/ipykernel_11144/25927682.py:37: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  helpers.bulk(client, docs, chunk_size=1000, request_timeout=200)
/var/folders/8h/pn2qk1n5571cghfklz699nh80000gn/T/ipykernel_11144/25927682.py:37: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(client, docs, chunk_size=1000, request_timeout=200)
/var/folders/8h/pn2qk1n5571cghfklz699nh80000gn/T/ipykernel_11144/25927682.py:37: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  helpers.bulk(client, docs, chunk_size=1000, request_timeout=200)


(3, [])

In [55]:
query = {
    "query": {
        "bool": {
            "must": [
                {"match": {"title": "cabbage"}},
            ],
            "filter": [
                {"match": {"type": "seed"}},
                {"range": {"price": {"gte":50 }}}
            ]
        }
    }
}

result = client.search(index=ELASTIC_SEARCH_INDEX, body=query)

hits = result.body.get('hits')
result_items = hits.get('hits')
result_items

/var/folders/8h/pn2qk1n5571cghfklz699nh80000gn/T/ipykernel_11144/466214737.py:15: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = client.search(index=ELASTIC_SEARCH_INDEX, body=query)
/var/folders/8h/pn2qk1n5571cghfklz699nh80000gn/T/ipykernel_11144/466214737.py:15: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = client.search(index=ELASTIC_SEARCH_INDEX, body=query)


[{'_index': 'test-farm-inputs',
  '_type': 'FARM_PRODUCT',
  '_id': '31ffd66c-bc8f-4983-8d5b-266e59e304f7',
  '_score': 0.5908618,
  '_source': {'type': 'seed',
   'title': 'CABBAGE',
   'price': 80,
   'timestamp': '2022-07-03T14:13:10.527616'}},
 {'_index': 'test-farm-inputs',
  '_type': 'FARM_PRODUCT',
  '_id': '3df78730-896d-4866-8255-0ca0d24353cb',
  '_score': 0.4700036,
  '_source': {'type': 'seed',
   'title': 'CABBAGE - BETTER',
   'price': 100,
   'timestamp': '2022-07-03T14:13:10.527631'}}]